# Importing Libary and Reading Dataset

In [1]:
import pandas as pd
import numpy as np

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import Binarizer

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier

In [27]:
df=pd.read_csv('train.csv')[['Age','Fare','SibSp','Parch','Survived']]
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [28]:
df.isnull().sum()

Age         177
Fare          0
SibSp         0
Parch         0
Survived      0
dtype: int64

In [29]:
age_mean=df['Age'].mean()
df.fillna(age_mean,inplace=True)

In [30]:
df.isnull().sum()

Age         0
Fare        0
SibSp       0
Parch       0
Survived    0
dtype: int64

In [31]:
df.sample(5)

,Age,Fare,SibSp,Parch,Survived
78,0.83,29.0000,0,2,1
842,30.00,31.0000,0,0,1
818,43.00,6.4500,0,0,0
631,51.00,7.0542,0,0,0
465,38.00,7.0500,0,0,0


In [33]:
df['family']=df['SibSp']+df['Parch']

In [34]:
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [35]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [36]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [39]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [43]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [45]:
x_train.head()

,Age,Fare,family
331,45.5,28.5000,0
733,23.0,13.0000,0
382,32.0,7.9250,0
704,26.0,7.8542,1
813,6.0,31.2750,6


# Without Binarization

In [46]:
clf=DecisionTreeClassifier()

clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [47]:
y_pred=clf.predict(x_test)
accuracy_score(y_pred,y_test)

0.6703910614525139

In [48]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.6397627965043695

# Applying Binarization

In [49]:
trf=ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])#by default Threshold is set to 0 and here we need threshold to be 0 so thats why we are not specifing it 
],remainder='passthrough')

In [50]:
x_train_trf=trf.fit_transform(x_train)
x_test_trf=trf.transform(x_test)

In [54]:
pd.DataFrame(x_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,0.0,45.500000,28.5000
1,0.0,23.000000,13.0000
2,0.0,32.000000,7.9250
3,1.0,26.000000,7.8542
4,1.0,6.000000,31.2750
...,...,...,...
707,0.0,21.000000,7.6500
708,0.0,29.699118,31.0000
709,1.0,41.000000,14.1083
710,1.0,14.000000,120.0000


In [53]:
clf=DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred1=clf.predict(x_test_trf)

accuracy_score(y_pred1,y_test)

0.6815642458100558

In [55]:
x_trf=trf.fit_transform(x)

In [57]:
np.mean(cross_val_score(DecisionTreeClassifier(),x_trf,y,scoring='accuracy',cv=10))

0.6554806491885143